<a href="https://colab.research.google.com/github/PinmanHuang/MLG/blob/master/LinkPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


# Upload and Read Data



In [3]:
# upload file into colab
from google.colab import files
uploaded = files.upload()

Saving content.csv to content.csv
Saving test.csv to test.csv
Saving train.csv to train.csv
Saving upload.csv to upload.csv


In [0]:
# read upload csv

# edge_id, to_node, from_node, node_attrs, edge_label
import pandas as pd

# attributes data
data_attrs = pd.read_csv("content.csv", sep='\t', header=None, index_col = 0)
data_attrs = data_attrs.sort_index(axis=0)  # sorting data in index
node_attrs = torch.tensor(data_attrs.values)

# train data
data_train = pd.read_csv("train.csv", sep=',')
train_edge_id = data_train[['id']].values.tolist()
train_to_node = data_train[['to']].values.tolist()
train_from_node = data_train[['from']].values.tolist()
train_edge_label = data_train[['label']].values.tolist()

# test data
data_test = pd.read_csv("test.csv", sep=',')
test_edge_id = data_train[['id']].values.tolist()
test_to_node = data_train[['to']].values.tolist()
test_from_node = data_train[['from']].values.tolist()

In [5]:
data_attrs.head(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
data_train.head(5)

,id,to,from,label
0,E10311,2399,2339,0
1,E10255,2397,1144,1
2,E10667,854,1726,0
3,E9395,872,702,0
4,E5926,2450,1312,1


In [7]:
data_test.head(5)

,id,to,from
0,E10559,2323,2673
1,E4849,81,1634
2,E3964,2405,1765
3,E542,2114,498
4,E331,1013,849


# Data


In [0]:
# create training X
X = torch.zeros([len(train_edge_id), node_attrs.size()[1]*2], dtype=torch.long)
for f_node, t_node, i in zip(train_from_node, train_to_node, range(len(train_edge_id))):
    X[i] = torch.cat((node_attrs[f_node], node_attrs[t_node]), dim=1)

# create training Y
Y = torch.zeros([len(train_edge_id), 1], dtype=torch.float32)
for label, i in zip(train_edge_label, range(len(train_edge_id))):
    Y[i][0] = label[0]

# Model

In [0]:
class Net(torch.nn.Module):
    def __init__(self, D_in = 2866, E = 1, H_sizes = [2048, 2048, 1024, 64, 32, 8], D_out = 1):
        super(Net, self).__init__()

        self.D_in = D_in
        self.E = E
        self.H_sizes = H_sizes
        self.D_out = D_out

        self.embedding = torch.nn.Embedding(self.D_in, self.E)
        self.fc1 = torch.nn.Linear(self.D_in, self.H_sizes[0], bias=True)
        self.hidden = torch.nn.ModuleList()
        for i in range(len(self.H_sizes)-1):
            self.hidden.append(torch.nn.Linear(self.H_sizes[i], self.H_sizes[i+1], bias=True))
        self.fc2 = torch.nn.Linear(self.H_sizes[-1], self.D_out, bias=True)

    def forward(self, x):
        x_emd = self.embedding(x)
        x_emd = torch.squeeze(x_emd, 2)
        h1 = self.fc1(x_emd)
        h = torch.sigmoid(h1)
        for layer in self.hidden:
            h = layer(h)
            h = torch.sigmoid(h)
        h2 = self.fc2(h)
        y_pre = torch.sigmoid(h2)
        return y_pre

# Training

In [0]:
# Training Parameters
EPOCHS = 100
LEARNING_RATE = 0.002

In [22]:
# parameters
D_in = 2866
E = 1
H_sizes = [4096, 8192, 4096, 2048, 1024, 64, 32, 16, 8]
D_out = 1
# model = Net()
model = Net(D_in, E, H_sizes, D_out).to(device)
print(model)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Net(
  (embedding): Embedding(2866, 1)
  (fc1): Linear(in_features=2866, out_features=4096, bias=True)
  (hidden): ModuleList(
    (0): Linear(in_features=4096, out_features=8192, bias=True)
    (1): Linear(in_features=8192, out_features=4096, bias=True)
    (2): Linear(in_features=4096, out_features=2048, bias=True)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): Linear(in_features=1024, out_features=64, bias=True)
    (5): Linear(in_features=64, out_features=32, bias=True)
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)


In [18]:
# X = X-0.5
X = X.to(device)
Y = Y.to(device)
for i in range(EPOCHS):
    
    optimizer.zero_grad()

    y_pred = model(X)
    loss = torch.nn.functional.binary_cross_entropy(y_pred, Y)
    print("EPOCH #{}'s loss: {}".format(i+1, loss))
    loss.backward()
    optimizer.step()

EPOCH #1's loss: 0.7053122520446777
EPOCH #2's loss: 0.7035725116729736
EPOCH #3's loss: 0.7021152973175049
EPOCH #4's loss: 0.7008042931556702
EPOCH #5's loss: 0.6996092796325684
EPOCH #6's loss: 0.6985250115394592
EPOCH #7's loss: 0.6975499987602234
EPOCH #8's loss: 0.6966829895973206
EPOCH #9's loss: 0.6959220170974731
EPOCH #10's loss: 0.695264458656311
EPOCH #11's loss: 0.6947067379951477
EPOCH #12's loss: 0.6942448616027832
EPOCH #13's loss: 0.6938745379447937
EPOCH #14's loss: 0.6935890913009644
EPOCH #15's loss: 0.6933819055557251
EPOCH #16's loss: 0.6932440996170044
EPOCH #17's loss: 0.6931663155555725
EPOCH #18's loss: 0.6931381821632385
EPOCH #19's loss: 0.6931493878364563
EPOCH #20's loss: 0.6931890845298767
EPOCH #21's loss: 0.6932473182678223
EPOCH #22's loss: 0.6933143734931946
EPOCH #23's loss: 0.6933819055557251
EPOCH #24's loss: 0.6934436559677124
EPOCH #25's loss: 0.6934942007064819
EPOCH #26's loss: 0.6935302019119263
EPOCH #27's loss: 0.6935506463050842
EPOCH #28's

## Batch

In [0]:
import torch.utils.data as Data
BATCH_SIZE = 4096
torch_dataset = Data.TensorDataset(X, Y)
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

In [24]:
for i in range(EPOCHS):
    
    for step, (batch_x, batch_y) in enumerate(loader):
        optimizer.zero_grad()
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        y_pred = model(batch_x)
        loss = torch.nn.functional.binary_cross_entropy(y_pred, batch_y)
        print("EPOCH #{}'s batch{} loss: {}".format(i+1, step, loss))
        loss.backward()
        optimizer.step()

EPOCH #1's batch0 loss: 0.6993962526321411
EPOCH #1's batch1 loss: 0.6945520043373108
EPOCH #1's batch2 loss: 0.6986106038093567
EPOCH #2's batch0 loss: 0.6968657970428467
EPOCH #2's batch1 loss: 0.6934847235679626
EPOCH #2's batch2 loss: 0.696308434009552
EPOCH #3's batch0 loss: 0.6950867176055908
EPOCH #3's batch1 loss: 0.6929751038551331
EPOCH #3's batch2 loss: 0.6945996284484863
EPOCH #4's batch0 loss: 0.6938709616661072
EPOCH #4's batch1 loss: 0.6929461359977722
EPOCH #4's batch2 loss: 0.6934503316879272
EPOCH #5's batch0 loss: 0.6931660175323486
EPOCH #5's batch1 loss: 0.6932787895202637
EPOCH #5's batch2 loss: 0.6927840113639832
EPOCH #6's batch0 loss: 0.6928614974021912
EPOCH #6's batch1 loss: 0.6937844753265381
EPOCH #6's batch2 loss: 0.6924736499786377
EPOCH #7's batch0 loss: 0.6927996873855591
EPOCH #7's batch1 loss: 0.694252610206604
EPOCH #7's batch2 loss: 0.6923658847808838
EPOCH #8's batch0 loss: 0.6928251385688782
EPOCH #8's batch1 loss: 0.6945315599441528
EPOCH #8's ba